In [1]:
import numpy as np
import time
import math
import itertools
import requests
from bs4 import BeautifulSoup
import sympy
import copy
import xlwings as xw
from sympy.abc import a

In [2]:
# I found this stackexchange post very informative
# https://math.stackexchange.com/questions/3934750/how-to-solve-this-candy-collectors-puzzle
# and the book referred to https://www2.math.upenn.edu/~wilf/DownldGF.html is vey good.

print("Puzzle")
print("~~~~~~")
url='https://www.janestreet.com/puzzles/figurine-figuring/'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
x =[text for text in soup.body.stripped_strings]
print("".join(x[7:13]))

Puzzle
~~~~~~
Jane received 78 figurines as gifts this holiday season:  12 drummers drumming, 11 pipers piping, 10 lords a-leaping, etc., down to 1 partridge in a pear tree.   They are all mixed together in a big bag.  She agrees with her friend Alex that this seems like too many figurines for one person to have, so she decides to give some of her figurines to Alex.   Jane will uniformly randomly pull figurines out of the bag one at a time until she pulls out the partridge in a pear tree, and will give Alex all of the figurines she pulled out of the bag (except the partridge, that’s Jane’s favorite).Ifnis the maximum number of any one type of ornament that Alex gets, what is the expected value ofn, to seven significant figures?


<hr>
<b>Generating function is :</b>
<br>
<br>
    $\Large\prod_{j=2}^i\left(\sum_{k=0}^j \frac{j!}{(j-k)!} \frac{a^k}{k!}\right)$

In [3]:
# Generating function using sympy and put the coeffs into an array
# remove higher order terms to find probabilites with constraints on 
# max of any one figure

perms=np.zeros((12,78))

for cap in range(1,13):
    gfa = sympy.poly(1,a)
    for j in range(2,13):
        gfb = sympy.poly(0,a)
        for k in range(0,min(j+1,cap+1)):
            gfb += sympy.poly((math.factorial(j)/math.factorial(j-k))*(a**k)/math.factorial(k),a)
        gfa *=gfb
    print("With a max of {} of any ball you can draw {}".format(cap,len(gfa.monoms())-1))    
    perms[cap-1,78-len(gfa.coeffs()):]=gfa.coeffs()

print(np.array(perms).shape)

With a max of 1 of any ball you can draw 11
With a max of 2 of any ball you can draw 22
With a max of 3 of any ball you can draw 32
With a max of 4 of any ball you can draw 41
With a max of 5 of any ball you can draw 49
With a max of 6 of any ball you can draw 56
With a max of 7 of any ball you can draw 62
With a max of 8 of any ball you can draw 67
With a max of 9 of any ball you can draw 71
With a max of 10 of any ball you can draw 74
With a max of 11 of any ball you can draw 76
With a max of 12 of any ball you can draw 77
(12, 78)


In [4]:
# Checking a simple case
# - expectation for 2 pulls longhand
# - then extract from the matrix of coeffs to check

grid = np.zeros((11,11))

for i,j in itertools.product(range(11),range(11)):
    grid[i,j] = (i+2)/77*(j+2-(i==j))/76
    
print("expectation for 2 pulls      =",1+np.sum(grid * np.eye(11)))

x = perms[0,-3] # ways of pulling 2 with a cap of 1 => x/y is expectation of 1
y = perms[1,-3] # ways of pulling 2 with a cap of 2 => (y-x)/y is expectation of 2

print("From the generating function =",((2*(y-x)+x)/y))

expectation for 2 pulls      = 1.0977443609022557
From the generating function = 1.0977443609022557


In [5]:
# Generalizing the above for the whole matrix
# lopping off the last column as it represents 0 draws
# 1 is equally likely to be drawn at every point so divide by 78

processed = np.zeros((12,78))
processed[0,:]=perms[0,:]
for i in range(11,0,-1):
    processed[i,:] = perms[i,:] - perms [i-1,:]
processed = (processed/np.sum(processed,axis=0))[:,:77]*np.array([np.arange(1,13)]).T

print("Expectation is : {:.6f}".format(np.sum(processed)/78))

Expectation is : 6.859787


In [6]:
url='https://www.janestreet.com/puzzles/solutions/january-2021-solution/'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
x =[text for text in soup.body.stripped_strings]

print(" ".join(x[7:9]))

This month’s puzzle required a careful computation to remain tractable.  One approach that would work: by keeping track of the position of the partridge figurine and the maximum count of identical figurines drawn before it as the twelve different sets of figurines were inserted into the permutation reduced the space into a matrix of size 78 x 12.  The final expected value was approximately 6.859787


In [8]:
# Check with a simulation to see if I'm close

urn = np.array(
      [1,
       2,2,
       3,3,3,
       4,4,4,4,
       5,5,5,5,5,
       6,6,6,6,6,6,
       7,7,7,7,7,7,7,
       8,8,8,8,8,8,8,8,
       9,9,9,9,9,9,9,9,9,
       10,10,10,10,10,10,10,10,10,10,
       11,11,11,11,11,11,11,11,11,11,11,
       12,12,12,12,12,12,12,12,12,12,12,12],dtype=int)

def loop():
    temp = copy.copy(urn)
    np.random.shuffle(temp)
    picks = np.zeros(11)
    for draw in temp:
        if draw == 1:
            return np.max(picks)
        else:
            picks[draw-2] +=1
            
start = time.perf_counter()
sum = 0
for i in range(10000000):
    sum += loop()
    if i % 1000000 ==0:
        print(sum/(i+1))
        
stop =  time.perf_counter()
print('\n Solution took {:0.4f} seconds\n'.format((stop-start)))

9.0
6.85959514040486
6.859817070091465
6.858856713714429
6.859196785200804
6.858591228281754
6.858106190315635
6.859182305831099
6.859125392609326
6.859076237880418

 Solution took 433.4376 seconds

